# Predição de sobreviventes do Titanic com Regressão Logística
## Introdução

Nesse notebook, vou utilizar um algoritmo de regressão logística para prever sobreviventes do acidente do Titanic. Para isso, vou:
1. Ler e separar os dados em dataset de treino e teste;
2. Tratar e limpar os dados;
3. Realizar uma análise exploratória para obter insights e compreender quais features devo utilizar no meu algoritmo;
4. Treinar o modelo e otimizá-lo;
5. Prever sobreviventes a partir do dataset de teste e comparar com as respostas reais.
<br>

Vamos lá!

## Descrição das variáveis 

Survival : 0 se não sobreviveu e 1 se sobreviveu

Pclass : Classe do quarto do passageiro: 1st = Alta, 2nd = Média, 3rd = Baixa

SibSp : Número de irmãos e cônjuges

Parch : Número de pais ou filhos

Ticket : Número do ticket

Fare : Tarifa do passageiro

Cabin : Número da cabine do passageiro

Embarked: Aonde o passageiro embarcou. C = Cherbourg, Q = Queenstown, S = Southampton

Name: nome de passageiro

Sex: gênero do passageiro

Age: idade do passageiro

## Bibliotecas utilizadas

In [75]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import scipy
import sklearn

## Lendo Dados

In [76]:
data = pd.read_csv('Data/train.csv', index_col='PassengerId')
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [77]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


## Separando entre Dataset de Treino e Teste

In [78]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=42)
print("Tamanho do train de treino: ", train.shape)
print("Tamanho do train de teste: ", test.shape)

Tamanho do train de treino:  (712, 11)
Tamanho do train de teste:  (179, 11)


# Análise Exploratória dos Dados

In [79]:
cols = list(train.columns)
cols

['Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [80]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 332 to 103
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  712 non-null    int64  
 1   Pclass    712 non-null    int64  
 2   Name      712 non-null    object 
 3   Sex       712 non-null    object 
 4   Age       572 non-null    float64
 5   SibSp     712 non-null    int64  
 6   Parch     712 non-null    int64  
 7   Ticket    712 non-null    object 
 8   Fare      712 non-null    float64
 9   Cabin     159 non-null    object 
 10  Embarked  710 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 66.8+ KB


É possível verificar que as colunas "Age", "Cabin" e "Embarked" possuem valores nulos. Vou verificar qual percentual de valores nulos para concluir se é interessante substituí-los ou simplesmente remover a coluna em questão.

In [81]:
def null_count(train):
    null_count = train.isnull().sum()
    null_count = null_count[null_count > 0] / len(train) * 100
    null_count.sort_values(inplace=True)
    return null_count

null_count(train)

Embarked     0.280899
Age         19.662921
Cabin       77.668539
dtype: float64

"Cabin" possue 77.6% de valores nulos, então vou descartá-la. Após analisar individualmente as outras duas colunas vou conseguir concluir qual melhor opção para substituir os valores nulos.

In [82]:
cols.remove("Cabin")

In [83]:
num_cols = list(train.select_dtypes(exclude=["object"]).columns) # colunas numéricas
obj_cols = list(train.select_dtypes(include=["object"]).columns) # colunas não-numéricas
obj_cols.remove("Cabin")
num_cols.remove("Survived")

### Colunas Numéricas

In [84]:
train[num_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 332 to 103
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  712 non-null    int64  
 1   Age     572 non-null    float64
 2   SibSp   712 non-null    int64  
 3   Parch   712 non-null    int64  
 4   Fare    712 non-null    float64
dtypes: float64(2), int64(3)
memory usage: 33.4 KB


In [85]:
def multiple_plots(df, graph_type, cols=list(train.columns), n_cols=5):
    """Retorna um objeto Figure (plotly) com um gráfico para cada coluna especificada
    """
    n_rows = int(np.ceil(len(cols) / n_cols))
    fig = make_subplots(n_rows, n_cols, subplot_titles=cols)
    
    i = 1
    j = 1
    for i in range(1, n_rows + 1):
        for j in range(1, n_cols + 1):
            col = cols[(i - 1) * n_cols + j - 1]
            if graph_type == 'hist':
                fig.add_trace(go.Histogram(x=df[col], name=col), row=i, col=j)
            elif graph_type == 'box':
                fig.add_trace(go.Box(y=df[col], name=""), row=i, col=j)
            elif graph_type == 'violin':
                fig.add_trace(go.Violin(y=df[col], name=""), row=i, col=j)
                    
    fig.update_layout(height=400, width=1000)
    return fig
    

fig = multiple_plots(train, "box", num_cols, 5)
fig.update_layout(title_text="Box Plots das colunas numéricas")
fig.show()

É possível verificar que todas colunas numéricas (menos "Pclass") possuem outliers. Vou exibir um histograma para cada coluna para visualizar melhor esses outliers quando comparados com o resto da distribuição.

In [86]:
fig = multiple_plots(train, "hist", num_cols)
fig.update_layout(title_text="Histogramas das colunas numéricas")
fig.show()

É possível verificar que em especial "Fare" possue muitos outliers. <br>
Agora, qual a relação entre essas variáveis e a taxa de sobrevivência dos passageiros? Para isso, vou comparar a taxa média de sobrevivência para cada grupo das variáveis em análise.

In [87]:
fig = make_subplots(rows=1, cols=5, subplot_titles=num_cols)

for idx, col in enumerate(num_cols):
    if col in ["Age", "Fare"]:
        grouped_df = train[[col, "Survived"]].groupby(col).mean()
        fig.add_trace(go.Scatter(x=grouped_df.index, y=grouped_df.Survived, name=col, mode="markers"), row=1, col=idx + 1)
    else:
        grouped_df = train[[col, "Survived"]].groupby(col).mean()
        fig.add_trace(go.Bar(x=grouped_df.index, y=grouped_df.Survived, name=col), row=1, col=idx + 1)
    
fig.update_layout(height=400, width=1000, title_text="Média da taxa de sobrevivência em relação com cada coluna numérica")

fig.show()

Aparentemente, verifica-se uma relação decrescente quanto a taxa de sobrevivência em relação a "Pclass", e crescente em relação a "Parch". Vou analisar individualmente as colunas para realizar uma análise mais profunda.

#### Idade

In [88]:
fig = px.histogram(train, x="Age", 
                   color="Survived", color_discrete_sequence=["#363945", "#B6E880"])
fig.update_layout(height=400, width=800, title_text="Sobrevivência em relação à idade")
fig.show()

#### Tarifa paga

In [89]:
fig = px.histogram(train, x="Fare", color="Survived", nbins=20,
                   color_discrete_sequence=["#363945", "#B6E880"])
fig.update_layout(height=400, width=800, title_text="Sobrevivência em relação à tarifa paga")
fig.show()

In [90]:
fig = px.box(train, y="Fare", color="Survived", color_discrete_sequence=["#363945", "#B6E880"])
fig.update_layout(height=400, width=800, title_text="Sobrevivência em relação à tarifa paga")
fig.show()

Aparentemente, pessoas que pagaram mais caro pela passagem sobreviveram mais. Apesar de haver muitos outliers, estão presentes tanto com "Survived" == 1 quanto com "Survived" == 0. Mas há um outlier muito extremo ("Fare" ~= 512). Vou removê-lo.

In [91]:
train = train[train.Fare < 512]

In [92]:
fig = px.box(train, y="Fare", color="Survived", color_discrete_sequence=["#363945", "#B6E880"])
fig.update_layout(height=400, width=800, title_text="Sobrevivência em relação à tarifa paga")
fig.show()

Agora ficou mais evidente como aqueles que pagaram mais tiveram uma taxa de sobrevivência maior.

#### Classe do passageiro

In [93]:
fig = px.histogram(train, x="Pclass", color="Survived", color_discrete_sequence=["#363945", "#B6E880"])
fig.update_layout(height=400, width=800, title_text="Sobrevivência em relação à classe do passageiro")
fig.show()

As classes mais altas (1 e 2) apresentaram uma taxa de sobrevivência muito superior comparada a mais baixa (3). Será que há uma relação entre o preço da passagem e a classe do passageiro?

In [96]:
fig = px.box(train, x="Pclass", y="Fare")
fig.update_layout(height=400, width=800, title_text="Preço da passagem em relação a classe do passageiro")
fig.show()

Há uma relação muito forte! Não há necessidade então de utilizar ambas colunas. Como "Pclass" na realidade é uma variável categórica, vou mantê-la e remover "Fare". Mas para isso, preciso alterar o tipo da coluna "Pclass" para ser tratada como objeto, e não numérica.

In [98]:
train["Pclass"] = train["Pclass"].astype("category")
cols.remove("Fare")

#### nº de familiares

In [102]:
fig = px.histogram(train, x="SibSp", color="Survived", color_discrete_sequence=["#363945", "#B6E880"])
fig.update_layout(height=400, width=800, title_text="Sobrevivência em relação à quantidade de irmãos/cônjuges")
fig.show()

In [101]:
fig = px.histogram(train, x="Parch", color="Survived", color_discrete_sequence=["#363945", "#B6E880"])
fig.update_layout(height=400, width=800, title_text="Sobrevivência em relação à quantidade de pais/filhos")
fig.show()

As duas variáveis em análise acima possuem um comportamento muito semelhante. Vou adicioná-las em apenas uma coluna que representa o número de familiares.

In [104]:
train["n_family"] = train["SibSp"] + train["Parch"]
cols.remove("SibSp")
cols.remove("Parch")
cols.append("n_family")

Agora vamos analisar as colunas categóricas.

### Colunas categóricas

In [105]:
train[obj_cols].head()

,Name,Sex,Ticket,Embarked
PassengerId,,,,
332,"Partner, Mr. Austen",male,113043,S
734,"Berriman, Mr. William John",male,28425,S
383,"Tikkanen, Mr. Juho",male,STON/O 2. 3101293,S
705,"Hansen, Mr. Henrik Juul",male,350025,S
814,"Andersson, Miss. Ebba Iris Alfrida",female,347082,S


"Name" não é uma coluna categórica, então vou removè-la. Mas observando seu conteúdo, cada passageiro possue um título pelo qual é chamado ("Mr", "Miss", etc.). Será que esse título tem alguma relação com a taxa de sobrevivência?

In [107]:
cols.remove("Name")
obj_cols.remove("Name")

In [119]:
def get_title(name):
    name = name.split(" ")
    title = name[1]
    if title in ["Mr.", "Mrs.", "Miss.", "Master."]:
        return title
    else:
        return "Other"

In [120]:
train["title"] = train.Name.map(get_title)
train.title.value_counts()

Mr.        404
Miss.      139
Mrs.        92
Other       41
Master.     33
Name: title, dtype: int64

In [122]:
fig = px.histogram(train, x="title", color="Survived", color_discrete_sequence=["#363945", "#B6E880"])
fig.update_layout(height=400, width=800, title_text="Sobrevivência em relação a como o passageiro é chamado (título)")
fig.show()

Passageiros chamados de "Mr." viveram muito menos do que outros.

In [125]:
cols.append("title")

In [110]:
train[obj_cols].describe()

,Sex,Ticket,Embarked
count,709,709,707
unique,2,557,3
top,male,CA. 2343,S
freq,465,7,525


Analisando a tabela acima, "Ticket" possue muitos valores únicos (assim como "Name"). Vou removê-la também.

In [111]:
cols.remove("Ticket") # too many unique values

#### Gênero

In [113]:
fig = px.histogram(train, x="Sex", color="Survived", color_discrete_sequence=["#363945", "#B6E880"])
fig.update_layout(height=400, width=800, title_text="Sobrevivência por gênero")
fig.show()

É evidente que mulheres sobreviveram muito mais que homens. Aparentemente gênero é uma feature muito importante para a predição de sobrevivência de um passageiro.

#### Onde embarcou

In [123]:
fig = px.histogram(train, x="Embarked", color="Survived", color_discrete_sequence=["#363945", "#B6E880"])
fig.update_layout(height=400, width=800, title_text="Sobrevivência por porto de embarque")
fig.show()

Pessoas que embarcaram em Southampton sobreviveram com menos frequência.

## Preparando dados para o modelo.

In [126]:
train[cols].head()

,Survived,Pclass,Sex,Age,Embarked,n_family,title
PassengerId,,,,,,,
332,0,1,male,45.5,S,0,Mr.
734,0,2,male,23.0,S,0,Mr.
383,0,3,male,32.0,S,0,Mr.
705,0,3,male,26.0,S,1,Mr.
814,0,3,female,6.0,S,6,Miss.


In [127]:
cat_cols = ["Sex", "Embarked", "title"]
num_cols = ["n_family", "Pclass", "Age"]

X = train[cols]
y = train["Survived"]

In [128]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy="most_frequent")),
    ('encoder', OneHotEncoder(handle_unknown="ignore")),
])

num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('scaler', StandardScaler())
])

data_prep_pipe = ColumnTransformer([
    ("cat_cols", cat_pipe, cat_cols),
    ("num_cols", num_pipe, num_cols)
])

X_prep = data_prep_pipe.fit_transform(X)
X_prep.shape

(709, 13)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, f1_score, mean_absolute_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb

In [ ]:
def get_model(name, params=None):
    if params == None:
        if name == "logreg":
            return LogisticRegression(random_state=42)
        elif name == "KNN":
            return KNeighborsClassifier()
        elif name == "RF":
            return RandomForestClassifier(random_state=42)
        elif name == "SVC":
            return SVC(random_state=42)
        elif name == "XGBoost":
            return xgb.XGBClassifier(random_state=42, verbosity=0, use_label_encoder=False)
        elif name == "LightGBM":
            return lgb.LGBMClassifier(random_state=42)
    else:
        if name == "logreg":
            return LogisticRegression(**params)
        elif name == "KNN":
            return KNeighborsClassifier(**params)
        elif name == "RF":
            return RandomForestClassifier(**params)
        elif name == "SVC":
            return SVC(**params)
        elif name == "XGBoost":
            return xgb.XGBClassifier(**params)
        elif name == "LightGBM":
            return lgb.LGBMClassifier(**params)

In [ ]:
list_models = ["logreg", "KNN", "RF", "SVC", "XGBoost", "LightGBM"]

for model_name in list_models:
    model = get_model(model_name)
    preds = cross_val_predict(model, X_prep, y, cv=10)
    f1 = f1_score(y, preds)
    roc_auc = roc_auc_score(y, preds)
    mae = mean_absolute_error(y, preds)
    print(f"{model_name}:\nF1: {f1:.3f},\nROC AUC: {roc_auc:.3f},\nMAE: {mae:.3f}\n")


In [ ]:
list_models.remove("RF")
list_models.remove("KNN")

In [ ]:
list_models

In [ ]:
def get_params_dist(name):
    if name == "logreg":
        return {"random_state": [42], 
                "solver": ["lbfgs", "liblinear"],
                "C" : [100, 10, 1.0, 0.1, 0.01]}
    elif name == "SVC":
        return {"random_state": [42], 
                "kernel": ["rbf", "linear"],
                'C': [1e0, 1e1, 1e2, 1e3], 
                "gamma": [0.5, 0.6, 0.7, 0.8, 0.9]}
    elif name == "XGBoost":
        return {"random_state": [42], "verbosity": [0], "use_label_encoder": [False],
                'max_depth': [3,6,10],
                'learning_rate': [0.01, 0.05, 0.1],
                'n_estimators': [100, 500, 1000],
                'colsample_bytree': [0.3, 0.7, 1.0],
                }
    elif name == "LightGBM":
        return {"random_state": [42],
                "max_depth": [3,6,10],
                "learning_rate": [0.01, 0.05, 0.1],
                "n_estimators": [100, 500, 1000],
                }

In [ ]:
for model_name in list_models:
    model = get_model(model_name)
    params_dist = get_params_dist(model_name)
    grid = GridSearchCV(model, params_dist, cv=5, scoring="roc_auc")
    grid.fit(X_prep, y)
    print(f"{model_name}:\nBest params: {grid.best_params_}\nBest score: {grid.best_score_}\n")

In [ ]:
best_model_name = "XGBoost"
model = get_model(best_model_name)
params_dist = get_params_dist(best_model_name)
grid = GridSearchCV(model, params_dist, cv=10, scoring="roc_auc")
grid.fit(X_prep, y)
preds = grid.predict(X_prep)
roc_auc = roc_auc_score(y, preds)
print(f"{best_model_name}:\nBest params: {grid.best_params_}\nROC AUC: {roc_auc:.3f}\n")

In [ ]:
import xgboost as xgb

params = grid.best_params_

xgb_model = xgb.XGBClassifier(**params)
train_preds = cross_val_predict(xgb_model, X_prep, y, cv=50, verbose=1)

In [ ]:
roc_auc = roc_auc_score(y, train_preds)
mae = mean_absolute_error(y, train_preds)
f1 = f1_score(y, train_preds)

print(f"ROC AUC: {roc_auc:.3f}")
print(f"MAE: {mae:.3f}")
print(f"F1: {f1:.3f}")

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y, preds, normalize='true', labels=[0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=[0, 1])
disp.plot() 
plt.show()

In [ ]:
fpr, tpr, thresholds = roc_curve(y, train_preds)

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    
plot_roc_curve(fpr, tpr)
plt.show()

# Train on Full Data

In [ ]:
full_model = get_model("XGBoost", params=params)
full_model.fit(X_prep, y)

X_test = test_data
X_test["title"] = X_test.Name.map(get_title)
X_test["FamilySize"] = X_test.Parch + X_test.SibSp
X_test["Age_class"] = X_test["Age"] // 15 * 15
X_test["Alone"] = X_test.FamilySize.map(lambda x: 1 if x == 0 else 0)
X_test = X_test[cols]

## Test Predictions

In [ ]:
X_test_prep = data_prep_pipe.transform(X_test)
test_preds = full_model.predict(X_test_prep)

## Submission

In [ ]:
# Saving the output for submission
output = pd.DataFrame({'PassengerId': test_data.index, 'Survived':test_preds})
output.to_csv('my_submission.csv', index=False)

In [ ]:
### How Should Performance be Measured?

data.Survived.value_counts()

# There is a significant amount of positive class values. Let's use as performance measure the ROC AUC score.